## 캐글 사용을 위한 설정

In [0]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 69 Dec 14 10:51 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=6679880b9d1fc4fb5cbdb54b06e5ef8c6dbb1945216efbf7a158b2fb4672458c
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

Kaggle API 1.5.6


## 데이터 셋 다운로드 후 압축 풀기

In [0]:
! kaggle competitions download -c 2019-ml-finalproject

import zipfile
import os

os.mkdir('/content/input2')

 65% 42.0M/65.0M [00:00<00:00, 117MB/s] 
100% 65.0M/65.0M [00:00<00:00, 244MB/s]


In [0]:
zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input2")
zip_ref.close()

## SIFT 사용을 위한 설정

In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 380kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 9.1MB/s 


## 라이브러리 로드

In [0]:
from imutils import paths
import numpy as np
import imutils 
import cv2 
import os
import pandas as pd
from tqdm import tqdm_notebook

from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from scipy.cluster.vq import *
import math

## 학습 데이터 준비

In [0]:
df_train = pd.read_csv("/content/input2/Label2Names.csv", header=None)

## SIFT를 이용하여 feature 검출

In [0]:
def dense_sift_each(img_gray):
  sift = cv2.xfeatures2d.SIFT_create()

  keypoints = []
  w, h = np.array(img_gray).shape

  for i in range(4, h, 8): # 4에서 h까지 8 step씩
    for j in range(4, w, 8):
      keypoints.append(cv2.KeyPoint(i, j, 8))
  kp, des = sift.compute(img_gray, keypoints)

  return kp, des

In [0]:
data_root_train = "/content/input2/train/"

train_images = []
labels = []
des_list = []
            
# sift 선언
sift = cv2.xfeatures2d.SIFT_create()

for i in tqdm_notebook(os.listdir(data_root_train)):
  img_cls_path = data_root_train + i + "/"  # 이미지 클래스 path
  img_path = [img_cls_path + j for j in os.listdir(img_cls_path)]  # 각 클래스 내의 학습이미지 path

  if i == "BACKGROUND_Google":       # Label2Names.csv에 빠져있는 카테고리 label 지정
    label = 0
  else:
    label = (df_train.index[df_train[1]==i] + 1).tolist()[0]  # 나머지 label 지정

  for img in img_path:
    image = cv2.imread(img)
    image = cv2.resize(image, (256,256))
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    train_images.append(image_gray)

    kp, des = dense_sift_each(image_gray)
    des_list.append(des)
    labels.append(label)

In [0]:
des_tot = np.vstack((descriptor for descriptor in des_list))
des_tot.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


(3133440, 128)

## K-Means Clustering을 이용하여 Code Book 만들기

In [0]:
pip install kmc2

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 9.4MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252221 sha256=51094fe35c219801a3e53f73e8f9308d8d494970a172cd101dcecb4713fb67b8
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import kmc2
from sklearn.cluster import MiniBatchKMeans

codebooksize = 800

In [0]:
seeding = kmc2.kmc2(np.array(des_tot).reshape(-1,128), codebooksize)
Kmeas = MiniBatchKMeans(codebooksize, init=seeding).fit(np.array(des_tot).reshape(-1,128))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=800. Setting it to 3*k
  init_size=init_size)


In [0]:
codeBook = Kmeas.cluster_centers_
print(codeBook.shape)
print(codeBook)

(800, 128)
[[42.383717  27.399557  19.473911  ... 16.760885  15.018845  29.023542 ]
 [11.5133295  7.7394185  8.332114  ... 10.82383   10.067164   6.18215  ]
 [42.374     73.43568   30.665087  ... 14.723938  12.884679  11.253355 ]
 ...
 [ 5.684539   4.451108  11.052574  ...  4.006839   6.7660336  6.3543625]
 [13.766121  36.433434  10.726092  ... 80.79502   20.584322   6.7517285]
 [ 4.68764    1.7062088  1.6712857 ... 20.414625  25.059593  32.8293   ]]


## Descriptor Pyramid 만들기

In [0]:
def input_vector_encoder(des, codeBook):
  code, _ = vq(des, codeBook)
  code_hist, _ = np.histogram(code, bins=range(codeBook.shape[0]+1))
  return code_hist

In [0]:
def build_spatial_pyramid(image, descriptor, level):
    step_size = 8
    height = int(image.shape[0] / step_size)    # 256/8 = 32 : des 개수
    width = int(image.shape[1] / step_size)

    # descriptor의 인덱스를 표시하기 위함(0~1023을 32*32의 배열로 표현)
    idx_crop = np.array(range(len(descriptor))).reshape(height,width)  # descriptor의 길이(개수:32*32)의 범위를 배열로
    size = idx_crop.itemsize  # 배열의 각 요소의 바이트 사이즈 -> strides 구하는데 필요

    bh, bw = 2**(5-level), 2**(5-level)
    shape = (int(height/bh), int(width/bw), bh, bw) # level에 맞춰 이미지를 쪼개기(앞에 두 숫자가 이미지 쪼갠 갯수)
    strides = size * np.array([width*bh, bw, width, 1]) # 배열을 탐색 할 때 각 차원에서 단계별로 이동할 바이트 수
    
    crops = np.lib.stride_tricks.as_strided(idx_crop, shape=shape, strides=strides) # 이미지(여기선 des) 쪼개기

    # 각각의 쪼개진 이미지에서 descriptor index를 1차원으로 만듬
    des_idxs = [col_block.flatten().tolist() for row_block in crops for col_block in row_block]

    # 위에 인덱스를 이용하여 
    pyramid = []
    for idxs in des_idxs:
        pyramid.append(np.asarray([descriptor[idx] for idx in idxs]))   
    return pyramid

In [0]:
def spatial_pyramid_matching(image, descriptor, codebook, level):
    pyramid = []
    if level == 0:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)

        code = [input_vector_encoder(crop, codebook) for crop in pyramid]

        return np.asarray(code).flatten()
    if level == 1:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)
        pyramid += build_spatial_pyramid(image, descriptor, level=1)

        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        code_level_0 = 0.5 * np.asarray(code[0]).flatten()
        code_level_1 = 0.5 * np.asarray(code[1:]).flatten()

        return np.concatenate((code_level_0, code_level_1))
    if level == 2:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)
        pyramid += build_spatial_pyramid(image, descriptor, level=1)
        pyramid += build_spatial_pyramid(image, descriptor, level=2)

        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        code_level_0 = 0.25 * np.asarray(code[0]).flatten()
        code_level_1 = 0.25 * np.asarray(code[1:5]).flatten()
        code_level_2 = 0.5 * np.asarray(code[5:]).flatten()
        
        return np.concatenate((code_level_0, code_level_1, code_level_2))

##SPM Kernel(Histogram Intersection)
#### 히스토그램 유사도를 측정하는 함수(https://github.com/wihoho/Image-Recognition/blob/5dc8834dd204e36172815345f0abe5640a4a37ef/recognition/classification.py#L10) (SPM 커널 적용하는부분)

In [0]:
def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result

##학습 데이터 pyramid 쌓기

In [0]:
X = [spatial_pyramid_matching(train_images[i], des_list[i], codeBook, level=2) for i in range(len(train_images))]

In [0]:
X = np.array(X)
y = np.array(labels)

In [0]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
X = scale.fit_transform(X)

##데이터셋 서브 샘플링 & Train-Validation 나누기

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=0)

##SVM 분류기 설계

In [0]:
gramMatrix = histogramIntersection(X_train, X_train)
# SPM을 통해서 만든 feature의 유사도를 구해서 gramMatrix를 정의

In [0]:
gramMatrix.shape

(2754, 2754)

In [0]:
param_grid = {'kernel':['precomputed'], 'C':[0.001, 0.005, 0.01, 0.05, 0.1]}
svc = SVC(gamma='scale')
grid = GridSearchCV(svc, param_grid, cv=5)

%time grid.fit(gramMatrix, y_train)
print(grid.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


CPU times: user 14.5 s, sys: 44.7 ms, total: 14.6 s
Wall time: 14.6 s
{'C': 0.005, 'kernel': 'precomputed'}


## 테스트 데이터 준비

In [0]:
data_root_test = "/content/input2/testAll_v2/"

img_list = os.listdir(data_root_test)
des_list_test = []
test_images = []

img_path_test = [data_root_test + i for i in img_list]

for img in tqdm_notebook(img_path_test):
  image = cv2.imread(img)
  image = cv2.resize(image, (256,256))
  image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  test_images.append(image_gray)

  kp, des = dense_sift_each(image_gray)
  des_list_test.append(des)

##테스트 데이터 pyramid 쌓기

In [0]:
X_test = [spatial_pyramid_matching(test_images[i], des_list_test[i], codeBook, level=2) for i in range(len(test_images))]
X_test = np.array(X_test)

##학습한 모델을 이용하여 테스트 데이터의 라벨 구하기

In [0]:
predictMatrix = histogramIntersection(X_test, X_train)
# test셋과 train셋에 대한 유사도를 구한 predictMatrix를 구함

In [0]:
model = grid.best_estimator_
y_pred = model.predict(predictMatrix)

In [0]:
y_pred = y_pred.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
result = np.hstack([result_img_list,y_pred])

In [0]:
print(result)

[['image_1389.jpg' '19']
 ['image_0810.jpg' '51']
 ['image_1265.jpg' '19']
 ...
 ['image_0532.jpg' '87']
 ['image_0288.jpg' '30']
 ['image_0906.jpg' '1']]


##결과 파일로 저장하여 Kaggle 제출 준비

In [0]:
df = pd.DataFrame(result, columns=['id','Category'])
df.to_csv('result-jhhwang.csv', index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f result-jhhwang.csv -m "Junghyun Hwang"

100% 29.6k/29.6k [00:00<00:00, 98.2kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 